# Rotational dynamics Quickstart

The calculation of rotational dynamics proceeds in two steps: (i) obtaining molecular rotational energies, wave functions, and matrix elements of electromagnetic property tensors, and (ii) solving the time-dependent problem with external electromagnetic fields using field-free solutions in (i) as basis.

## Molecular field-free rotational states

In [1]:
from richmol.rot.molecule import Molecule, mol_tensor

There are three common ways to set up the molecule and its properties, using calculated or experimental data or mix of both:

1. Provide Cartesian coordinates of atoms and molecular property tensors, referring to the same coordinate frame, obtained, for example, from the quantum chemical calculations.

2. Provide spectroscopic constants ($A$, $B$, $C$, $D_{J}$, $D_{J,K}$ ...) and molecular property tensors in the coordinate frame of principal axes of inertia (PAI).

3. Provide spectroscopic constants ($A$, $B$, $C$, $D_{J}$, $D_{J,K}$ ...) while molecular property tensors calculated in some other coordinate frame (not PAI). Here Cartesian coordinates of atoms must also be provided and will be used by the program only for rotating the property tensors to PAI frame.

### Molecular geometry and property tensors

Here is an example for water molecule, using data obtained from a quantum chemical calculation

In [2]:
water = Molecule()

water.XYZ = ("bohr",
             "O",  0.00000000,   0.00000000,   0.12395915,
             "H",  0.00000000,  -1.43102686,  -0.98366080,
             "H",  0.00000000,   1.43102686,  -0.98366080)

# dipole moment (au)
water.dip = [0, 0, -0.7288]

# polarizability tensor (au)
water.pol = [[9.1369, 0, 0], [0, 9.8701, 0], [0, 0, 9.4486]]

print(water.XYZ) # print Cartesian coordinates (in Angstrom) and masses of atoms

[('O', 15.99491462, [ 0.        ,  0.        ,  0.06559636])
 ('H',  1.00782503, [ 0.        , -0.75726686, -0.52053092])
 ('H',  1.00782503, [ 0.        ,  0.75726686, -0.52053092])]


By default, calculations are carried out for the main isotopologue. To specity a non-standard isotope, put the corresponding isotope number next to the atom label, for example, "O18" for oxygen-18 or "H2" for deuterium

In [3]:
# example of D2O^{18}
D2O18 = Molecule()
D2O18.XYZ = ("bohr",
             "O18",  0.00000000,   0.00000000,   0.12395915,
             "H2",   0.00000000,  -1.43102686,  -0.98366080,
             "H2",   0.00000000,   1.43102686,  -0.98366080)

print(D2O18.XYZ) # print Cartesian coordinates (in Angstrom) and masses of atoms

[('O18', 17.99915961, [ 0.        ,  0.        ,  0.06559636])
 ('H2',  2.01410178, [ 0.        , -0.75726686, -0.52053092])
 ('H2',  2.01410178, [ 0.        ,  0.75726686, -0.52053092])]


You can also read/store the geometry from/to the XYZ file

In [4]:
water.store_xyz("water.xyz", "comment line")

water2 = Molecule()
water2.XYZ = "water.xyz"

print(water2.XYZ)

[('O', 15.99491462, [ 0.        ,  0.        ,  0.06559636])
 ('H',  1.00782503, [ 0.        , -0.75726686, -0.52053092])
 ('H',  1.00782503, [ 0.        ,  0.75726686, -0.52053092])]


The molecular frame embedding, i.e., the orientation of $x,y,z$ axes with respect to the molecule, can be defined using `frame` property.
The molecule properties `dip` for dipole moment and `pol` for polarizability tensor (and few others) will be dynamically rotated to a new coordinate frame whenever the latter has been changed

In [5]:
# change frame to inertial principal axes system (ipas)
water.frame = "ipas" # or equivalently water.frame="diag(inertia)"

print("coordinates\n", water.XYZ)
print("dipole moment\n", water.dip)
print("polarizability\n", water.pol)

print("inertia tensor\n", water.inertia) # must be diagonal in "ipas" frame

coordinates
 [('O', 15.99491462, [ 0.        ,  0.06559636,  0.        ])
 ('H',  1.00782503, [-0.75726686, -0.52053092,  0.        ])
 ('H',  1.00782503, [ 0.75726686, -0.52053092,  0.        ])]
dipole moment
 [ 0.     -0.7288  0.    ]
polarizability
 [[9.8701 0.     0.    ]
 [0.     9.4486 0.    ]
 [0.     0.     9.1369]]
inertia tensor
 [[ 0.61496953 -0.         -0.        ]
 [-0.          1.15588076 -0.        ]
 [-0.         -0.          1.7708503 ]]


Multiple frame operations can be combined together, for example, we can rotate to the inertial principal axes system and then permute the $x$ and $z$ axes

In [6]:
water.frame = "ipas"
water.frame = "zyx"

# or in one line
water.frame = "zyx,ipas"

print("coordinates\n", water.XYZ)
print("dipole moment\n", water.dip)
print("polarizability\n", water.pol)

print("inertia tensor\n", water.inertia) # must be diagonal in "ipas" frame

coordinates
 [('O', 15.99491462, [ 0.        ,  0.06559636,  0.        ])
 ('H',  1.00782503, [ 0.        , -0.52053092, -0.75726686])
 ('H',  1.00782503, [ 0.        , -0.52053092,  0.75726686])]
dipole moment
 [ 0.     -0.7288  0.    ]
polarizability
 [[9.1369 0.     0.    ]
 [0.     9.4486 0.    ]
 [0.     0.     9.8701]]
inertia tensor
 [[ 1.7708503  -0.         -0.        ]
 [-0.          1.15588076 -0.        ]
 [-0.         -0.          0.61496953]]


The principal axes system can be defined with respect to any rank-2 symmetric matrix. In many cases it is convenient to choose molecular frame such that polarizability tensor becomes diagonal, which can be done as following

In [7]:
water.frame = "diag(pol)" # can also be combined with axes permutations, e.g., "xzy,diag(pol)"

print("coordinates\n", water.XYZ)
print("dipole moment\n", water.dip)
print("polarizability\n", water.pol)


coordinates
 [('O', 15.99491462, [ 0.        ,  0.06559636,  0.        ])
 ('H',  1.00782503, [ 0.        , -0.52053092, -0.75726686])
 ('H',  1.00782503, [ 0.        , -0.52053092,  0.75726686])]
dipole moment
 [ 0.     -0.7288  0.    ]
polarizability
 [[9.1369 0.     0.    ]
 [0.     9.4486 0.    ]
 [0.     0.     9.8701]]


In principle, user can define custom rotation matrix or principal axes matrix for frame rotation, as in the example below

In [8]:
import numpy as np
import scipy

# random matrix
mat = np.random.rand(3,3)

# random rotation matrix
water.custom_rot = scipy.linalg.expm((mat - mat.T)/2)

# random symmetric matrix
water.custom_pam = (mat + mat.T)/2

# use 'custom_rot' (orthogonal) matrix to rotate frame
water.frame = "custom_rot"

# use 'custom_pam' (symmetric) matrix as principal axes matrix
water.frame = "diag(custom_pam)"

Multiple occurences of `frame` assignments lead to the accumulation of the corresponding frame rotations, to reset the frame to the initial one, as defined by the input Cartesian coordinates of atoms, use `frame=null`

In [9]:
water.frame = "diag(inertia)" # rotate to ipas
water.frame = "zxy" # then permute axes (123)

# now we want to permute axes (23) in the original frame, reset frame rotation
water.frame = "xzy,null"

print(water.XYZ)

[('O', 15.99491462, [ 0.        ,  0.06559636,  0.        ])
 ('H',  1.00782503, [ 0.        , -0.52053092, -0.75726686])
 ('H',  1.00782503, [ 0.        , -0.52053092,  0.75726686])]


As mentioned above, only certain Cartesian molecule properties (such as `dip`, `pol`, `inertia`, `XYZ`, and few others) are automatically rotated when the frame is altered. This is demonstrated in the following example

In [10]:
water.vec = [1,2,3]
water.mat = [[1,2,3],[4,5,6],[7,8,9]]

water.frame = "zyx,null"

# axes permutation (13) does not affect vec and mat, but dip
print("vec\n", water.vec)
print("mat\n", water.mat)
print("dip\n", water.dip)

vec
 [1, 2, 3]
mat
 [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
dip
 [-0.7288  0.      0.    ]


If you want to declare a custom Cartesian tensor which is dynamically rotated to a new frame, use `mol_tensor` function

In [14]:
water.vec = mol_tensor([1,2,3])
water.mat = mol_tensor([[1,2,3],[4,5,6],[7,8,9]])

water.frame = "zyx,null"

# axes permutation (13) now affects vec and mat
print("vec\n", water.vec)
print("mat\n", water.mat)
print("dip\n", water.dip)

# we can then change vec and mat values, they will be still dynamically rotated
water.vec = [7,8,9]
water.mat = mol_tensor([[11,12,13],[14,15,16],[17,18,19]])

print("new vec\n", water.vec)
print("new mat\n", water.mat)

vec
 [3. 2. 1.]
mat
 [[9. 8. 7.]
 [6. 5. 4.]
 [3. 2. 1.]]
dip
 [-0.7288  0.      0.    ]
new vec
 [9. 8. 7.]
new mat
 [[19. 18. 17.]
 [16. 15. 14.]
 [13. 12. 11.]]
